In [12]:
import pandas as pd
datapath = "../data/amazon_reviews/amazon_appliances_reviews.csv"
df = pd.read_csv(datapath)
texts = df['review_body'].tolist()

In [13]:
import re
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def some_cleaning(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r'\d+','',text)
    text = [word.lower() for word in text.split() if word.lower() not in stop_words]
    text = [word for word in text if len(word) > 3]
    return text

texts = [some_cleaning(text) for text in texts]

In [14]:
# ensure that we have no empty strings, and we also want to filter out very short texts
texts = [text for text in texts if len(text) > 3]

In [17]:
from gensim.corpora.dictionary import Dictionary

gensim_dict = Dictionary(texts)
corpus = [gensim_dict.doc2bow(text) for text in texts]

In [19]:
# create an LSI model
from gensim.models import LsiModel
N_TOPICS = 3
lsi = LsiModel(corpus=corpus, num_topics=N_TOPICS, id2word=gensim_dict)

In [20]:
for topic in lsi.show_topics(num_topics=3, num_words=10):
    topic_id, topic_word_weights = topic
    words = topic_word_weights.split("+")
    w = [word.strip().replace('"', '').split("*")[1] for word in words]
    print(w)
    print("__")

['water', 'machine', 'washer', 'would', 'like', 'clothes', 'time', 'unit', 'cycle', 'dont']
__
['washer', 'water', 'machine', 'clothes', 'fridge', 'unit', 'cycle', 'oven', 'door', 'wash']
__
['water', 'fridge', 'machine', 'dryer', 'freezer', 'service', 'door', 'level', 'warranty', 'oven']
__


In [21]:
lsi.show_topics(num_topics=10, num_words=10)

[(0,
  '0.310*"water" + 0.232*"machine" + 0.210*"washer" + 0.197*"would" + 0.164*"like" + 0.162*"clothes" + 0.158*"time" + 0.142*"unit" + 0.126*"cycle" + 0.121*"dont"'),
 (1,
  '0.348*"washer" + 0.319*"water" + 0.315*"machine" + 0.298*"clothes" + -0.260*"fridge" + -0.223*"unit" + 0.159*"cycle" + -0.145*"oven" + -0.145*"door" + 0.144*"wash"'),
 (2,
  '-0.540*"water" + -0.397*"fridge" + 0.367*"machine" + 0.210*"dryer" + -0.138*"freezer" + 0.136*"service" + -0.116*"door" + -0.114*"level" + 0.104*"warranty" + 0.101*"oven"')]

In [22]:
from gensim.models import LdaModel

lda = LdaModel(corpus=corpus,num_topics=N_TOPICS)
for topic in lda.show_topics(num_topics=N_TOPICS, num_words=10, formatted=False):
    word_ids = [word for word, _ in topic[1]]
    words = [gensim_dict[int(word_id)] for word_id in word_ids]
    print(words)

['machine', 'water', 'washer', 'dishwasher', 'clothes', 'dryer', 'cycle', 'clean', 'wash', 'dishes']
['great', 'fridge', 'unit', 'good', 'well', 'like', 'would', 'wine', 'works', 'price']
['would', 'service', 'unit', 'years', 'oven', 'product', 'months', 'warranty', 'back', 'problem']


In [23]:
some_other_text = "10/10 I absolutely love this thing! It dries my clothes in 2 microseconds!"
some_other_text = some_cleaning(some_other_text)
bow = gensim_dict.doc2bow(some_other_text)
topic_matches = lda.get_document_topics(bow)
best_topic = max(topic_matches, key=lambda x: x[1])
print(f"Predicted topic {best_topic[0]} with probability {best_topic[1]}")

matching_words = [gensim_dict[int(word_id)] for word_id, _ in lda.show_topic(best_topic[0], topn=10)]
print(matching_words)

Predicted topic 0 with probability 0.879182755947113
['machine', 'water', 'washer', 'dishwasher', 'clothes', 'dryer', 'cycle', 'clean', 'wash', 'dishes']


In [29]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
gensimvis.prepare(lda, corpus, gensim_dict)

/opt/anaconda3/lib/python3.9/site-packages/joblib/numpy_pickle.py:230: UserWarning: The memmapped array [[-14.13975186 -11.01262454 -11.20798578]
 [ -9.83386612  -8.84326557  -7.71929277]
 [ -6.78911005  -5.94715111  -6.76982636]
 ...
 [-15.09424225 -15.32287929 -13.54295227]
 [-13.95010499 -15.49622936 -15.27564127]
 [-13.95010499 -15.49622936 -15.27564127]] loaded from the file /var/folders/0k/1bg63zt532nb9d86g5tk_6vh0000gn/T/joblib_memmapping_folder_48029_e5abf40973da4239b675a85819315d72_f477b585555f4bac982e7659994f59b5/48029-11231565136-d55f75c39db54215ad6a78395d11f7ea.pkl is not not bytes aligned. This may cause segmentation faults if this memmapped array is used in some libraries like BLAS or PyTorch. To get rid of this warning, regenerate your pickle file with joblib >= 1.2.0. See https://github.com/joblib/joblib/issues/563 for more details
  warnings.warn(message)
/opt/anaconda3/lib/python3.9/site-packages/joblib/numpy_pickle.py:230: UserWarning: The memmapped array [[-14.13975

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.147896  0.003325       1        1  39.285156
0      0.070839 -0.120250       2        1  31.324422
1      0.077058  0.116925       3        1  29.390422, topic_info=        Term          Freq         Total Category  logprob  loglift
72   machine  20580.000000  20580.000000  Default  30.0000  30.0000
50    washer  18018.000000  18018.000000  Default  29.0000  29.0000
68   clothes  12333.000000  12333.000000  Default  28.0000  28.0000
34     dryer  11812.000000  11812.000000  Default  27.0000  27.0000
125    water  26052.000000  26052.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
16      love   4483.871065   7492.049788   Topic3  -5.4800   0.7111
145   really   5182.528386  11806.937325   Topic3  -5.3351   0.4011
44      much   5291.393820  12989.709065   Topic3  -5.3144   0.3264
125    water   6333.421665  26052.937033   Topic3  -5.1346  -0.1898
127     also   4626.075621  13898.032284   Topic3  -5.4487   0.1245

[253 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
864       2  0.999133  agitator
127       1  0.279608      also
127       2  0.387537      also
127       3  0.332853      also
4828      3  0.998707   ambient
...     ...       ...       ...
203       3  0.085161      year
419       1  0.742944     years
419       2  0.221746     years
419       3  0.035335     years
1142      3  0.999053      zone

[442 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])